## Part 1: Data Loading and Cleaning

In [46]:

%pip install pandas 
import pandas as pd
import re
from datetime import datetime

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [47]:
# Saving data file path 
file_path= r'C:\Users\bansa\Downloads\calgary_access_log'  

In [48]:
# Apache-style log pattern through regex

#data ex.- local - - [24/Oct/1994:13:41:41 -0600] "GET index.html HTTP/1.0" 200 150

log_pattern = re.compile(
    r'(?P<host>\S+) \S+ \S+ \[(?P<timestamp>[^\]]+)\] "(?P<method>\S+)? (?P<resource>\S+)? (?P<protocol>[^"]+)?" (?P<status>\d{3}) (?P<bytes>\S+)'
)           #syntax (?P<name>...)


# Timestamp parser
#converting dateTime from time into date time object of pandas
def parse_timestamp(time):
    try:
        return datetime.strptime(time.split()[0], "%d/%b/%Y:%H:%M:%S")
    except:
        return pd.NaT

In [49]:
#parsing through data and storing it in data frame
data = []

with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
    for line in file:
        match = log_pattern.match(line)
        if match:
            data_entry = match.groupdict()
            data_entry['timestamp'] = parse_timestamp(data_entry['timestamp'])
            data_entry['bytes'] = int(data_entry['bytes']) if data_entry['bytes'].isdigit() else 0
            data_entry['status'] = int(data_entry['status'])

            # Extract file extension from the requested resource
            extension_match = re.search(r'\.([a-zA-Z0-9]+)(?:[\?#]|$)', data_entry.get("resource") or "")
            data_entry['file_ext'] = extension_match.group(1).lower() if extension_match else None

            #if these 3 colns are  present in any row then only we will append that data_entry 
            if all([data_entry['timestamp'], data_entry['method'], data_entry['resource']]):
            
                data.append(data_entry)


In [50]:
# Converting  it into DataFrame
df = pd.DataFrame(data)

# Drop malformed entries if any

required_cols = ['timestamp', 'method', 'resource']                       #list of column names that are necessary for analysis

df.dropna(subset=required_cols, inplace=True)         #drop row if there is "NA" in these required_cols



# Type casting
#making sure that data types are correct
df = df.astype({
    "host": str,
    "method": str,
    "resource": str,
    "protocol": str,
    "status": int,
    "bytes": int,
    "file_ext": "object"
})

df.reset_index(drop=True, inplace=True)

# Preview cleaned data
df.head()

,host,timestamp,method,resource,protocol,status,bytes,file_ext
0,local,1994-10-24 13:41:41,GET,index.html,HTTP/1.0,200,150,html
1,local,1994-10-24 13:41:41,GET,1.gif,HTTP/1.0,200,1210,gif
2,local,1994-10-24 13:43:13,GET,index.html,HTTP/1.0,200,3185,html
3,local,1994-10-24 13:43:14,GET,2.gif,HTTP/1.0,200,2555,gif
4,local,1994-10-24 13:43:15,GET,3.gif,HTTP/1.0,200,36403,gif


## Part 2: Analysis Questions

### Q1: Count of total log records

In [51]:
def total_log_records() -> int:
    """
    Q1: Count of total log records.

    Objective:
        Determine the total number of HTTP log entries in the dataset.
        Each line in the log file represents one HTTP request.

    Returns:
        int: Total number of log entries.
    """

    # TODO: Implement logic to count log records
    return len(df)

    return 0  # Placeholder return


answer1 = total_log_records()
print("Answer 1:")
print(answer1)

Answer 1:
724036


### Q2: Count of unique hosts

In [52]:
def unique_host_count() -> int:
    """
    Q2: Count of unique hosts.

    Objective:
        Determine how many distinct hosts accessed the server.

    Returns:
        int: Number of unique hosts.
    """

    # TODO: Implement logic to count unique hosts
    df_new=df["host"].nunique()
    return df_new

    return 0  # Placeholder return


answer2 = unique_host_count()
print("Answer 2:")
print(answer2)

Answer 2:
2


### Q3: Date-wise unique filename counts

In [53]:
def datewise_unique_filename_counts() -> dict[str, int]:
    """
    Q3: Date-wise unique filename counts.

    Objective:
        For each date, count the number of unique filenames that accessed the server.
        The date should be in 'dd-MMM-yyyy' format (e.g., '01-Jul-1995').

    Returns:
        dict: A dictionary mapping each date to its count of unique filenames.
              Example: {'01-Jul-1995': 123, '02-Jul-1995': 150}
    """

    # TODO: Implement logic for date-wise unique filename counts
    df['date']=df['timestamp'].dt.strftime('%d-%b-%Y')            #to get day-month-year format from dateTime
    count=df.groupby('date')['resource'].nunique().to_dict()      #for each date counting no of resources and then coverting result into dict
    return count
    return {}  # Placeholder return


answer3 = datewise_unique_filename_counts()
print("Answer 3:")
print(answer3)

Answer 3:
{'01-Apr-1995': 436, '01-Aug-1995': 672, '01-Dec-1994': 271, '01-Feb-1995': 622, '01-Jan-1995': 88, '01-Jul-1995': 387, '01-Jun-1995': 590, '01-Mar-1995': 582, '01-May-1995': 467, '01-Nov-1994': 412, '01-Oct-1995': 554, '01-Sep-1995': 328, '02-Apr-1995': 466, '02-Aug-1995': 857, '02-Dec-1994': 324, '02-Feb-1995': 524, '02-Jan-1995': 141, '02-Jul-1995': 399, '02-Jun-1995': 515, '02-Mar-1995': 600, '02-May-1995': 701, '02-Nov-1994': 427, '02-Oct-1995': 871, '02-Sep-1995': 351, '03-Apr-1995': 795, '03-Aug-1995': 584, '03-Dec-1994': 189, '03-Feb-1995': 569, '03-Jan-1995': 310, '03-Jul-1995': 438, '03-Jun-1995': 398, '03-Mar-1995': 505, '03-May-1995': 589, '03-Nov-1994': 460, '03-Oct-1995': 847, '03-Sep-1995': 213, '04-Apr-1995': 821, '04-Aug-1995': 717, '04-Dec-1994': 212, '04-Feb-1995': 561, '04-Jan-1995': 324, '04-Jul-1995': 612, '04-Jun-1995': 353, '04-Mar-1995': 403, '04-May-1995': 684, '04-Nov-1994': 404, '04-Oct-1995': 891, '04-Sep-1995': 342, '05-Apr-1995': 891, '05-Aug-19

### Q4: Number of 404 response codes

In [54]:
def count_404_errors() -> int:
    """
    Q4: Number of 404 response codes.

    Objective:
        Count how many times the HTTP 404 Not Found status appears in the logs.

    Returns:
        int: Number of 404 errors.
    """

    # TODO: Implement logic to count 404 errors
    count_404 = (df['status'] == 404).sum()       #filtering rows with status=404 and then keeping their count
    return count_404

    return 0  # Placeholder return


answer4 = count_404_errors()
print("Answer 4:")
print(answer4)

Answer 4:
23531


### Q5: Top 15 filenames with 404 responses

In [55]:
def top_15_filenames_with_404() -> list[tuple[str, int]]:
    """
    Q5: Top 15 filenames with 404 responses.

    Objective:
        Identify which requested URLs most frequently resulted in a 404 error.
        Return the top 15 filenames sorted by frequency.

    Returns:
        list: A list of tuples (filename, count), sorted by count in descending order.
              Example: [('index.html', 200), ...]
    """

    # TODO: Implement logic to find top 15 filenames with 404
    
    # Filtering rows with 404 status
    row_404 = df[df['status'] == 404]

    # from these filtered rows, Count frequency of each resource (filename) that caused a 404
    filename_counts = row_404['resource'].value_counts()   

    # Sort by frequency 
    filename_counts_sorted = filename_counts.sort_values(ascending=False).head(15)

    # Converting to list of tuples
    top_404_list = list(filename_counts_sorted.items())

    return top_404_list
    return []  # Placeholder return


answer5 = top_15_filenames_with_404()
print("Answer 5:")
print(answer5)

Answer 5:
[('index.html', 4737), ('4115.html', 902), ('1611.html', 649), ('5698.xbm', 585), ('710.txt', 408), ('2002.html', 259), ('2177.gif', 193), ('10695.ps', 161), ('6555.html', 153), ('487.gif', 152), ('151.html', 149), ('40.html', 148), ('488.gif', 148), ('3414.gif', 148), ('9678.gif', 142)]


### Q6: Top 15 file extension with 404 responses

In [56]:
def top_15_ext_with_404() -> list[tuple[str, int]]:
    """
    Q6: Top 15 file extensions with 404 responses.

    Objective:
        Find which file extensions generated the most 404 errors.
        Return the top 15 sorted by number of 404s.

    Returns:
        list: A list of tuples (extension, count), sorted by count in descending order.
              Example: [('html', 45), ...]
    """

    # TODO: Implement logic to find top 15 extensions with 404
    #filtering rows with status=404
    df_404 = df[df['status'] == 404]
    
    #dropping rows with missing file extension
    df_404_ext = df_404.dropna(subset=['file_ext'])

    #Sorting by frequency 
    top_404_extensions = df_404_ext['file_ext'].value_counts().head(15)
    return list(top_404_extensions.items())
    return []  # Placeholder return


answer6 = top_15_ext_with_404()
print("Answer 6:")
print(answer6)

Answer 6:
[('html', 12199), ('gif', 7340), ('xbm', 824), ('ps', 754), ('jpg', 538), ('txt', 508), ('htm', 109), ('cgi', 77), ('com', 45), ('z', 41), ('dvi', 40), ('ca', 36), ('hmtl', 30), ('util', 29), ('bmp', 28)]


### Q7: Total bandwidth transferred per day for the month of July 1995

In [57]:
def total_bandwidth_per_day() -> dict[str, int]:
    """
    Q7: Total bandwidth transferred per day for the month of July 1995.

    Objective:
        Sum the number of bytes transferred per day.
        Skip entries where the byte field is missing or '-'.

    Returns:
        dict: A dictionary mapping each date to total bytes transferred.
              Example: {'01-Jul-1995': 123456789, ...}
    """

    # TODO: Implement logic to compute total bandwidth per day
    
    #Filter entries from July 1995
    df_july95 = df[(df['timestamp'].dt.month == 7) & (df['timestamp'].dt.year == 1995)]

    #To Ensure bytes is a numeric field (already done earlier, but just in case)
    df_july95 = df_july95[df_july95['bytes'].notnull()]

    #Create a 'date' column in desired format: '01-Jul-1995'
    df_july95['date_str'] = df_july95['timestamp'].dt.strftime('%d-%b-%Y')

    #Group by date and sum the bytes
    bandwidth_per_day = df_july95.groupby('date_str')['bytes'].sum().to_dict()
    return bandwidth_per_day
    return {}  # Placeholder return


answer7 = total_bandwidth_per_day()
print("Answer 7:")
print(answer7)

Answer 7:
{'01-Jul-1995': 11333976, '02-Jul-1995': 8656012, '03-Jul-1995': 13596612, '04-Jul-1995': 26573988, '05-Jul-1995': 19541225, '06-Jul-1995': 19755015, '07-Jul-1995': 9427822, '08-Jul-1995': 5403491, '09-Jul-1995': 4660556, '10-Jul-1995': 14916848, '11-Jul-1995': 22503471, '12-Jul-1995': 17367065, '13-Jul-1995': 15988328, '14-Jul-1995': 19186430, '15-Jul-1995': 15773233, '16-Jul-1995': 9005564, '17-Jul-1995': 19601338, '18-Jul-1995': 17098855, '19-Jul-1995': 17851725, '20-Jul-1995': 20751717, '21-Jul-1995': 25455607, '22-Jul-1995': 8066660, '23-Jul-1995': 9593870, '24-Jul-1995': 22308265, '25-Jul-1995': 24550821, '26-Jul-1995': 24638042, '27-Jul-1995': 25969995, '28-Jul-1995': 36458881, '29-Jul-1995': 11696365, '30-Jul-1995': 23189598, '31-Jul-1995': 30729809}


### Q8: Hourly request distribution

In [58]:
def hourly_request_distribution() -> dict[int, int]:
    """
    Q8: Hourly request distribution.

    Objective:
        Count the number of requests made during each hour (00 to 23).
        Useful for understanding traffic peaks.

    Returns:
        dict: A dictionary mapping hour (int) to request count.
              Example: {0: 120, 1: 90, ..., 23: 80}
    """

    # TODO: Implement logic for hourly distribution
    df['hour'] = df['timestamp'].dt.hour            #extracting hour from dateTime

    #Count number of requests per hour and coverting into dict
    hourly_requests = df.groupby('hour').size().to_dict()         #.size() would give no of log entries in particular hour
    return hourly_requests
    return {}  # Placeholder return


answer8 = hourly_request_distribution()
print("Answer 8:")
print(answer8)

Answer 8:
{0: 18701, 1: 14372, 2: 12681, 3: 10895, 4: 9964, 5: 10787, 6: 13047, 7: 16659, 8: 26554, 9: 33966, 10: 43348, 11: 47570, 12: 46776, 13: 51405, 14: 54483, 15: 50269, 16: 51137, 17: 45047, 18: 33144, 19: 30546, 20: 29675, 21: 27392, 22: 23812, 23: 21806}


### Q9: Top 10 most requested filenames

In [59]:
def top_10_most_requested_filenames() -> list[tuple[str, int]]:
    """
    Q9: Top 10 most requested filenames.

    Objective:
        Identify the most commonly requested URLs (irrespective of status code).

    Returns:
        list: A list of tuples (filename, count), sorted by count in descending order.
                Example: [('index.html', 500), ...]
    """

    # TODO: Implement logic to find top 10 most requested filenames
    
    #Count frequency of each requested resource
    top_10_files = df['resource'].value_counts().head(10)
    top_10_files=top_10_files.sort_values(ascending=False)
    #Convert to list of tuples
    top_10_filenames = list(top_10_files.items())
    return top_10_filenames
    return []  # Placeholder return


answer9 = top_10_most_requested_filenames()
print("Answer 9:")
print(answer9)

Answer 9:
[('index.html', 140074), ('3.gif', 24006), ('2.gif', 23606), ('4.gif', 8018), ('244.gif', 5149), ('5.html', 5010), ('4097.gif', 4874), ('8870.jpg', 4493), ('6733.gif', 4278), ('8472.gif', 3843)]


### Q10: HTTP response code distribution

In [60]:
def response_code_distribution() -> dict[int, int]:
    """
    Q10: HTTP response code distribution.

    Objective:
        Count how often each HTTP status code appears in the logs.

    Returns:
        dict: A dictionary mapping HTTP status codes (as int) to their frequency.
              Example: {200: 150000, 404: 3000}
    """

    # TODO: Implement logic for response code counts

    #taking count of each status unique value and then coverting in dict format
    status_code_count = df['status'].value_counts().to_dict()
    return status_code_count
    return {}  # Placeholder return


answer10 = response_code_distribution()
print("Answer 10:")
print(answer10)

Answer 10:
{200: 567551, 304: 97792, 302: 30275, 404: 23531, 403: 4743, 401: 46, 501: 43, 500: 42, 400: 13}
